In [1]:
'''
Editor: @Hyunhomo
Data source: https://www.phmsociety.org/events/conference/phm/18/data-challenge

This implementation can be used for convert industrial raw data log to run-to-fail data
DataFrame contains RUL(time-to-failure) with cycles
'''

## Import libraries in python
import os
import pandas as pd
import padasip as pa
# pa.filters.FilterRLS(n)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import importlib
from scipy.stats import randint, expon, uniform

import sklearn as sk
from sklearn import svm
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn import preprocessing
from sklearn import pipeline
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
# How many rows to read 
num_cycles = 4000000

## Load and assign training data which includes all sensor and other measurements(attributes)
df_01_M02_DC_train = pd.read_csv('Data/01_M02_DC_train.csv', nrows = num_cycles)
# pd.set_option('display.max_rows', 10000)
# df_01_M02_DC_train[80000:90000].head(10000)
# df_01_M02_DC_train


In [3]:
# How many rows to read 
num_cycles = 4000000

## Load and assign training data which includes all sensor and other measurements(attributes)
df_01_M02_DC_train = pd.read_csv('Data/01_M02_DC_train.csv', nrows = num_cycles)
pd.set_option('display.max_rows', 2000)

# df_01_M02_DC_train.head(2000)
df_01_M02_DC_train

'''
recipe
recipe_step
IONGAUGEPRESSURE : pressure reading for the main process chamber when under vacuum
ETCHSUPPRESSORCURRENT : ion current impacting the suppressor grid plate
FLOWCOOLFLOWRATE : rate of flow of helium through the flowcool circuit, controlled by mass flow controller
FLOWCOOLPRESSURE : resulting helium pressure in the flowcool circuit


'''

print (df_01_M02_DC_train['recipe'].unique())
print (df_01_M02_DC_train['recipe_step'].unique())


# df_01_M02_DC_train_reduced = df_01_M02_DC_train[['time','recipe', 'recipe_step', 'IONGAUGEPRESSURE', 
#                                                 'ETCHBEAMVOLTAGE', 'ETCHBEAMCURRENT','ETCHSUPPRESSORCURRENT',
#                                                  'ETCHGASCHANNEL1READBACK', 'FLOWCOOLFLOWRATE', 'FLOWCOOLPRESSURE']]
# df_01_M02_DC_train_reduced
# df_01_M02_DC_train_reduced.to_csv('Data/df_01_M02_DC_train_reduced.csv', index=False)


df_01_M02_DC_train_full = df_01_M02_DC_train

df_01_M02_DC_train_full.to_csv('Data/01_M02_DC_train_full.csv', index=False)
print ("csv file saved")

[67 68 66 73 69 71 72 70 74 76 77 75]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51  0 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94]
csv file saved


In [4]:
# df_01_M02_DC_train_reduced = pd.read_csv('Data/01_M02_DC_train_reduced.csv')
# df_01_M02_DC_train_reduced



In [5]:
# df_01_M02_DC_train = pd.read_csv('Data/ttf/01_M02_DC_train.csv', nrows = num_cycles, header=None,  index_col=False)
df_01_M02_DC_ttf = pd.read_csv('Data/ttf/01_M02_DC_train.csv', nrows = num_cycles)
pd.set_option('display.max_rows', 2000)
# print (df_01_M02_DC_ttf)
# df_01_M02_DC_ttf.head(2000)
# df_01_M02_DC_ttf[int(614786/5+36132/5+500)-500:int(614786/5+36132/5+500)+500]
# print (df_01_M02_DC_ttf)

c1 = df_01_M02_DC_ttf['TTF_FlowCool Pressure Dropped Below Limit'] == 0
c2 = df_01_M02_DC_ttf['TTF_Flowcool Pressure Too High Check Flowcool Pump'] == 0
c3 = df_01_M02_DC_ttf['TTF_Flowcool leak'] == 0

failure_idx = c1+c2+c3
print (failure_idx)

any_failure = df_01_M02_DC_ttf['time'].loc[failure_idx]
# print (time_FPDBL_failure)

any_failure_idx = any_failure.index.values
any_failure_time = any_failure.values
print (any_failure_idx)
print (any_failure_time)

FPDBL_index = np.where(c1)
FPTHC_index = np.where(c2)
FL_index = np.where(c3)

print ("FPDBL_index",FPDBL_index)
print ("FPTHC_index",FPTHC_index)
print ("FL_index",FL_index)



# del_idx = np.where(np.in1d(FPDBL_idx, [146084,146565,822821,822821,822933,823355,2693250,2893945]))[0]

# FPDBL_idx_ed = np.delete(FPDBL_idx, del_idx)
# FPDBL_time_ed = np.delete(FPDBL_time, del_idx)
# print (FPDBL_idx_ed)
# print (FPDBL_time_ed)


0          False
1          False
2          False
3          False
4          False
           ...  
3999995    False
3999996    False
3999997    False
3999998    False
3999999    False
Length: 4000000, dtype: bool
[ 131033  133014  144072  145953  146084  146565  816430  822332  822683
  822821  822933  823355  916683 2193412 2203555 2576444 2645858 2693250
 2827173 2893945 2899284 2915376 3000592 3761261 3761709]
[ 3907068  3916348  3962388  3970900  3971424  3973348  8072052  8123030
  8124434  8124986  8125434  8127122  9273100 18061382 18108796 22075508
 22575152 22793870 24521052 25002546 25025806 25136976 25636030 31425558
 31427350]
FPDBL_index (array([ 131033,  144072,  146084,  146565,  816430,  822332,  822821,
        822933,  823355, 2203555, 2645858, 2693250, 2827173, 2893945,
       2915376, 3000592]),)
FPTHC_index (array([2576444, 3761261, 3761709]),)
FL_index (array([ 133014,  145953,  822683,  916683, 2193412, 2899284]),)


/home/hyunhomo/anaconda3/lib/python3.7/site-packages/pandas/core/computation/expressions.py:194: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  op=op_str, alt_op=unsupported[op_str]


In [6]:
## Load fault data if needed
df_01_M02_train_fault_data = pd.read_csv('Data/01_M02_train_fault_data.csv', nrows = num_cycles)
# pd.set_option('display.max_rows', 1000)
# df_01_M02_train_fault_data


time_FPDBL_fault = df_01_M02_train_fault_data['time'].loc[df_01_M02_train_fault_data['fault_name'] == 'FlowCool Pressure Dropped Below Limit']
print (time_FPDBL_fault)

0       3907068
1       3915070
4       3918310
5       3937494
6       3940162
7       3958954
8       3962388
9       3968458
11      3971424
12      3973348
13      8059958
14      8072052
15      8105258
16      8123030
18      8124986
19      8125434
20      8127122
21      8132488
22      8136008
53     18057900
55     18074648
56     18097634
57     18108796
58     18139306
64     22575152
65     22779500
66     22793870
67     22828180
74     23630826
76     23834942
77     24521052
78     25002546
80     25136976
81     25636030
82     25637472
83     28029182
84     28113782
85     28126778
88     31427832
89     35337042
91     35479832
93     35667758
94     35672222
95     36022710
96     36132446
97     36134280
99     36140216
100    36145322
101    36402870
104    36712164
105    36765886
107    36777452
108    36797152
Name: time, dtype: int64


In [7]:
##
df_01_M02_DC_train_reduced = df_01_M02_DC_train_full

cols_to_use = df_01_M02_DC_ttf.columns.difference(df_01_M02_DC_train_reduced.columns)
df_train_ttf = pd.concat([df_01_M02_DC_train_reduced, df_01_M02_DC_ttf[cols_to_use]], axis=1)
# df_train = df_01_M02_DC_train_reduced
# Slicing training data for each failure cycle
# 
df_train_ttf_list = []
prev_idx = 0
# for f_idx in FPDBL_idx:
for cycle, f_idx in enumerate(any_failure_idx, start=1):
    print (cycle)
    print (prev_idx , f_idx)
    print (df_01_M02_DC_ttf['time'][prev_idx], df_01_M02_DC_ttf['time'][f_idx])
    cycle_array = np.zeros(f_idx-prev_idx+1, dtype=int)+cycle
    temp_df = df_train_ttf[prev_idx:f_idx+1] 
    temp_df['failure_cycle'] = cycle_array
    df_train_ttf_list.append(temp_df)
    prev_idx = f_idx +1


df_train_ttf_list[1]


1
0 131033
3283834 3907068
2
131034 133014
3907072 3916348
3
133015 144072
3916352 3962388
4
144073 145953
3962392 3970900
5
145954 146084
3970904 3971424
6
146085 146565
3971428 3973348
7
146566 816430
3973352 8072052


/home/hyunhomo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


8
816431 822332
8072056 8123030
9
822333 822683
8123034 8124434
10
822684 822821
8124438 8124986
11
822822 822933
8124990 8125434
12
822934 823355
8125438 8127122
13
823356 916683
8127126 9273100
14
916684 2193412
9273104 18061382
15
2193413 2203555
18061386 18108796
16
2203556 2576444
18108800 22075508
17
2576445 2645858
22075512 22575152
18
2645859 2693250
22575156 22793870
19
2693251 2827173
22793874 24521052
20
2827174 2893945
24521056 25002546
21
2893946 2899284
25002550 25025806
22
2899285 2915376
25025810 25136976
23
2915377 3000592
25136980 25636030
24
3000593 3761261
25636034 31425558
25
3761262 3761709
31425562 31427350


,time,Tool,stage,Lot,runnum,recipe,recipe_step,IONGAUGEPRESSURE,ETCHBEAMVOLTAGE,ETCHBEAMCURRENT,...,ACTUALROTATIONANGLE,FIXTURESHUTTERPOSITION,ETCHSOURCEUSAGE,ETCHAUXSOURCETIMER,ETCHAUX2SOURCETIMER,ACTUALSTEPDURATION,TTF_FlowCool Pressure Dropped Below Limit,TTF_Flowcool Pressure Too High Check Flowcool Pump,TTF_Flowcool leak,failure_cycle
131034,3907072,01M02,1,9361,10548007,67,43,-0.022398,1.189382,0.785026,...,-0.122455,1.0,2.787077,3.309812,3.631609,0.803526,7998.0,8523232.0,9276.0,2
131035,3907076,01M02,1,9361,10548007,67,43,-0.019429,1.188926,0.782897,...,-0.122455,1.0,2.787080,3.309815,3.631613,0.803526,7994.0,8523228.0,9272.0,2
131036,3907080,01M02,1,9361,10548007,67,43,-0.019429,1.188072,0.783535,...,-0.122455,1.0,2.787084,3.309819,3.631617,0.803526,7990.0,8523224.0,9268.0,2
131037,3907084,01M02,1,9361,10548007,67,43,-0.019429,1.187048,0.784953,...,-0.122455,1.0,2.787087,3.309823,3.631621,0.803526,7986.0,8523220.0,9264.0,2
131038,3907088,01M02,1,9361,10548007,67,43,-0.019429,1.187787,0.781764,...,-0.122455,1.0,2.787090,3.309827,3.631625,0.803526,7982.0,8523216.0,9260.0,2
131039,3907092,01M02,1,9361,10548007,67,43,-0.019429,1.186765,0.787792,...,-0.122455,1.0,2.787094,3.309830,3.631631,0.803526,7978.0,8523212.0,9256.0,2
131040,3907096,01M02,1,9361,10548007,67,43,-0.023038,1.187958,0.783750,...,-0.122455,1.0,2.787097,3.309834,3.631632,0.803526,7974.0,8523208.0,9252.0,2
131041,3907100,01M02,1,9361,10548007,67,43,-0.026130,1.188015,0.784104,...,-0.122455,1.0,2.787100,3.309838,3.631636,0.803526,7970.0,8523204.0,9248.0,2
131042,3907104,01M02,1,9361,10548007,67,43,-0.029054,1.188301,0.786017,...,-0.122455,1.0,2.787104,3.309841,3.631640,0.803526,7966.0,8523200.0,9244.0,2
131043,3907108,01M02,1,9361,10548007,67,43,-0.029054,1.189551,0.784530,...,-0.122455,1.0,2.787107,3.309845,3.631644,0.803526,7962.0,8523196.0,9240.0,2


In [8]:
df_train_ttf_merged = pd.concat(df_train_ttf_list)
df_train_ttf_merged

#Save to csv file
df_train_ttf_merged.to_csv('Data/01_M02_DC_train_ttf_merged.csv', index=False)
print ("csv file saved")

csv file saved
